# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine

engine = create_engine('sqlite:///DisasterResponse.db')
connection = engine.connect()
df = pd.read_sql_table(engine.table_names()[0], connection)

X = df['message']
Y = df.loc[:, df.columns != 'message']

In [3]:
Y.head(5)

,id,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
### index, id does not seem to have any factual implication 
### original seems to be replica of message but in other languages
### but what about genre? 
Y.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

<p>The value indicates the source where does the messaege derives from
It may seem to matter how importance/priority we treat the messages 
but if we treat each message with equal importance, then this message
may not seem important(the criteria for project seems to be classify responses to particular relief organization for handle</p>



In [5]:
### drop some columns that does not seem useful
# Y = Y.drop(columns=['index', 'id', 'original', 'genre'], axis=1)
Y = Y.drop(columns=[ 'id', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

text = "Dr. Smith graduated from the University of Washington. He later started an analytics firm called Lux, which catered to enterprise customers."
print(tokenize(text))

[nltk_data] Downloading package stopwords to /home/monki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/monki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/monki/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['dr', 'smith', 'graduated', 'university', 'washington', 'later', 'started', 'analytics', 'firm', 'called', 'lux', 'catered', 'enterprise', 'customer']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

#### To build a classifier to separate them into class
#### We try to choose from ensemble type of learning classifier, trees

pipeline = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# model = build_model()
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feat',
                 FeatureUnion(transformer_list=[('pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f6c53a9ef70>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

Category: related
              precision    recall  f1-score   support

           0       0.70      0.09      0.17      1266
           1       0.77      0.99      0.86      3938
           2       0.45      0.12      0.20        40

    accuracy                           0.76      5244
   macro avg       0.64      0.40      0.41      5244
weighted avg       0.75      0.76      0.69      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      4349
           1       0.77      0.48      0.59       895

    accuracy                           0.89      5244
   macro avg       0.83      0.73      0.76      5244
weighted avg       0.88      0.89      0.88      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.14      0.04      0.06        26

    accuracy                           0.99      5244
   macro avg       0.5

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
### get parameters we can tune the pipeline for
pipeline.get_params()

{'memory': None,
 'steps': [('feat', FeatureUnion(transformer_list=[('pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x7f6c53a9ef70>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'feat': FeatureUnion(transformer_list=[('pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f6c53a9ef70>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'feat__n_jobs': None,
 'feat__transformer_list': [('pipeline',
   Pipeline(steps=[('vec



Reference:
<ol>
    
<li>https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff</li>

<li>https://medium.datadriveninvestor.com/a-survey-of-evaluation-metrics-for-multilabel-classification-bb16e8cd41cd</li>
</ol>

A very important consideration to make for GridSearch is to consider the scoring it should use

In [11]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
    'clf__estimator__n_estimators': [10, 20, 40]
}

cv =  GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))


Category: related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1266
           1       0.75      1.00      0.86      3938
           2       0.00      0.00      0.00        40

    accuracy                           0.75      5244
   macro avg       0.25      0.33      0.29      5244
weighted avg       0.56      0.75      0.64      5244

Category: request
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      4349
           1       0.88      0.15      0.26       895

    accuracy                           0.85      5244
   macro avg       0.86      0.57      0.59      5244
weighted avg       0.86      0.85      0.80      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

/home/monki/anaconda3/envs/data_scientist/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
y_test.sum(axis=0)

related                   4018
request                    895
offer                       26
aid_related               2131
medical_help               422
medical_products           270
search_and_rescue          127
security                    88
military                   155
child_alone                  0
water                      339
food                       595
shelter                    470
clothing                    73
money                      104
missing_people              60
refugees                   171
death                      237
other_aid                  695
infrastructure_related     328
transport                  240
buildings                  267
electricity                122
tools                       32
hospitals                   46
shops                       22
aid_centers                 67
other_infrastructure       223
weather_related           1438
floods                     411
storm                      486
fire                        53
earthqua

### Try With Another Classifier
#### 1) We try wth another ensemble based classifier RandomForestClassifier

In [14]:
pipeline2 = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


In [15]:
pipeline2.fit(X_train,y_train)

y_pred = pipeline2.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

Category: related
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1266
           1       0.83      0.94      0.88      3938
           2       0.37      0.40      0.39        40

    accuracy                           0.81      5244
   macro avg       0.64      0.58      0.60      5244
weighted avg       0.80      0.81      0.79      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.50      0.62       895

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

/home/monki/anaconda3/envs/data_scientist/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.98      0.97      4758
           1       0.76      0.51      0.61       486

    accuracy                           0.94      5244
   macro avg       0.86      0.75      0.79      5244
weighted avg       0.93      0.94      0.93      5244

Category: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5191
           1       0.00      0.00      0.00        53

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

Category: earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4766
           1       0.87      0.77      0.81       478

    accuracy                           0.97      5244
   macro avg       0.92      0.88      0.90      5244
weighted avg       0.97      0.97     

In [16]:
#### 2) We try wth another ensemble based classifier XGBoostClassifier

In [17]:
pipeline3 = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
        ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])


In [20]:
y_train

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
23907,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
16852,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23022,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
860,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
15795,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
pipeline3.fit(X_train,y_train)

y_pred = pipeline3.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.

### 9. Export your model as a pickle file

In [ ]:
import pickle

filename = "classifier.pkl" 

with open(filename, 'wb') as file:  
    pickle.dump(pipeline,file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.